In [58]:
from eyecite import get_citations
from eyecite.models import FullCaseCitation, ShortCaseCitation
import polars as pl

In [59]:
df = pl.read_csv("./Cases/CourtListenerOpinions.csv")
fair_use = [c for c in df["Citation"]]

In [60]:
def get_fair_use_citations(doc, fair_use = fair_use):

    citations = [c.matched_text() for c in get_citations(doc) if type(c) is FullCaseCitation or type(c) is ShortCaseCitation]

    cited = set(citations).intersection(set(fair_use))

    return cited

In [61]:
df = df.with_columns(
    pl.col("Document").map_elements(get_fair_use_citations).alias("CitedCases")
)

In [62]:
df_citation = df.select(["CourtListenerCaseName", "CitedCases"])

In [63]:
citation_rel = df_citation.with_columns(
                pl.col("CitedCases").map_elements(list)
                ).explode("CitedCases").unique().sort("CourtListenerCaseName")

In [67]:
name_citation_map = df.select(["CourtListenerCaseName", "Citation"]).unique().rename({"CourtListenerCaseName": "CitedCaseName"})

In [71]:
citation_rel.rename({"CitedCases": "Citation"}).join(name_citation_map, on = ["Citation"], how = "inner").write_csv("./Cases/CitationRelationship.csv")